In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    "../../data/SP500_EGS_Score_avarage_per_year.csv",
    index_col=0,
    parse_dates=True,
    sep=",",
)

In [ ]:
df.head(20)

In [ ]:
# all lower case

df.columns = map(str.lower, df.columns)

# - to _

df.columns = df.columns.str.replace("-", "_")

In [ ]:
len(df.company_symbol.unique())

In [ ]:
import sys

sys.path.append("../")

In [ ]:
from helpers.text_preprocessing.preprocess_text import preprocess_text

In [ ]:
import pandas as pd

# Load or create your DataFrame (replace 'your_data.csv' with the path to your data file)
df = pd.read_csv(
    "../data/extracted_text_sustainability_reports.csv",
    index_col=0,
    parse_dates=True,
    sep=",",
)

In [ ]:
# Preprocess the text and get the preprocessed DataFrame
preprocessed_df = preprocess_text(df)

In [ ]:
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import plotly
from gensim import corpora
import mlflow

In [ ]:
import sys

sys.path.append("../")

In [ ]:
from models.LDA_optuna_tuning.tune_lda_optuna import train_lda, compute_coherence
from models.LDA_optuna_tuning.call_optuna_tune import (
    preprocess_data,
    execute_optuna_study,
)

In [ ]:
df = pd.read_csv(
    "../data/preprocessed_data_text_format.csv", index_col=0, parse_dates=True, sep=","
)

In [ ]:
df = df.head(200)

In [ ]:
# write the preprocessed data to csv
df.to_csv("../data/preprocessed_data_text_format_200.csv")

In [ ]:
preprocessed_stats = {
    "Number of Missing Values": df["preprocessed_content"].isna().sum(),
    "Number of Unique Texts": df["preprocessed_content"].nunique(),
    "Sample Texts": df["preprocessed_content"].sample(5).tolist(),
}

preprocessed_stats

In [ ]:
df

In [ ]:
if mlflow.active_run():
    mlflow.end_run()
execute_optuna_study(df, n_trials=200)

In [ ]:
import yfinance as yf

In [ ]:
# Create a ticker object
tesla = yf.Ticker("TSLA")

'''# To fetch row names from financials and put them in a list
financials_rows = tesla.financials.index.tolist()
print("Financials Rows:", financials_rows)

# To fetch row names from cashflow and put them in a list
cashflow_rows = tesla.cashflow.index.tolist()
print("Cashflow Rows:", cashflow_rows)

# To fetch row names from balance sheet and put them in a list
balance_rows = tesla.balance_sheet.index.tolist()
print("Balance Rows:", balance_rows)

print(tesla.info)'''
print(tesla.get_sustainability())

In [ ]:
import pandas as pd
from collections import Counter

df = pd.read_csv(
    "../data/ready_to_model/ready_to_model_df.csv",
    index_col=0,
    parse_dates=True,
    sep=",",
)

In [ ]:
# Filter out any non-string values in the 'preprocessed_content' column
df["preprocessed_content"] = df["preprocessed_content"].apply(
    lambda x: x if isinstance(x, str) else ""
)

# Tokenize the 'preprocessed_content' and flatten the list
all_words = [word for content in df["preprocessed_content"] for word in content.split()]

# Count the frequency of each word
counter = Counter(all_words)

# Display the 10 most common and 10 least common words
most_common_words = counter.most_common(300)
least_common_words = [word for word, count in counter.items() if count <= 2]


print("Most common words", most_common_words)
print("Least common words", least_common_words)
print(len(most_common_words))
print(len(least_common_words))

In [2]:
from sklearn.model_selection import train_test_split, KFold

import tpot2
import sklearn
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("../data/ready_to_model/df_cleaned_with_yfinance.csv")
df_sample = df.sample(100, random_state=100)

In [3]:
df = pd.read_csv("../data/ready_to_model/df_filtered_feature_importance.csv")


In [4]:
df

,Unnamed: 0,tfidf_svd_dim_4,637,134,69,287,83,38,balance_Additional Paid In Capital,balance_Net PPE,...,353,svd_dim_14,svd_dim_101,32,cashflow_Long Term Debt Issuance,balance_Accounts Payable,balance_Current Debt,695,svd_dim_50,total_score
0,0,-0.084914,-0.984096,0.268918,0.107418,-0.046604,-1.011947,-0.375769,0.000000e+00,2.815400e+09,...,-0.384085,-0.382449,0.050529,-0.734844,1.000000e+09,6.213000e+08,1500000.0,0.935146,-0.036939,15.30
1,1,0.011697,-0.408780,0.058946,-0.211146,-0.258639,-1.149698,0.105433,6.617404e+09,1.161558e+09,...,0.259594,0.004463,-0.062224,-0.494972,0.000000e+00,2.166409e+09,59482000.0,0.983424,0.011989,27.66
2,2,-0.045409,0.379066,0.062867,-0.268551,-0.470384,-0.945055,1.394683,3.160000e+06,9.470280e+08,...,-0.082250,-0.999183,0.172444,-0.779211,NaN,2.012461e+09,NaN,0.749328,-0.801159,18.13
3,3,0.057648,-0.725314,0.736935,-0.064104,-0.223336,-0.799952,0.159469,NaN,NaN,...,-1.051099,-0.039436,-0.029329,-0.679992,NaN,NaN,NaN,0.727001,0.598416,18.25
4,4,0.012955,-0.828527,-0.378939,-0.294139,0.299100,-1.169436,-0.405682,1.254200e+09,6.876000e+08,...,1.061694,1.390666,-0.036480,-0.481583,9.894000e+08,1.484000e+08,NaN,0.349011,-1.085175,21.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,844,0.026311,-0.129050,0.170490,-0.054193,-0.399259,-0.675335,0.057946,9.537000e+09,NaN,...,-0.103423,-0.755071,-0.001880,-0.450927,5.330000e+09,NaN,NaN,0.514164,-0.469090,21.70
845,845,-0.015391,-0.068218,0.163940,0.009989,-0.350510,-1.147584,0.601176,1.581000e+09,1.837400e+10,...,-0.156433,-0.379208,-0.044857,-0.130985,0.000000e+00,1.574000e+09,819000000.0,1.275327,0.384824,30.41
846,846,0.035103,-0.597744,-0.269980,0.086464,-0.059405,-0.315838,0.366545,NaN,NaN,...,-0.020172,0.668182,0.141643,-0.240015,NaN,NaN,NaN,0.807168,0.151807,19.73
847,847,-0.002000,-0.430859,-0.107497,0.018429,-0.108159,-0.717960,0.478447,1.434203e+09,2.342330e+08,...,-0.025405,0.027800,0.109576,-0.545853,3.750000e+08,1.869100e+07,NaN,1.088519,0.030471,15.40


In [ ]:
df.head()

In [5]:
'''"e_score",
"s_score",
"g_score",
"unnamed: 0",
"filename",
"ticker",
"year",
"preprocessed_content",
"ner_entities",
"company_symbol",'''

# columns to drop
columns_to_drop = [

    "total_score",
    "Unnamed: 0"
]

# Separate features and target
y = df["total_score"]
X = df.drop(columns=columns_to_drop)

In [6]:
# drop the last two rows
X = X.iloc[:-2, :]

In [7]:
y = y.iloc[:-2]

In [8]:
X.drop(columns=["Unnamed: 0"], inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [9]:
X

,tfidf_svd_dim_4,637,134,69,287,83,38,balance_Additional Paid In Capital,balance_Net PPE,tfidf_svd_dim_1,...,svd_dim_85,353,svd_dim_14,svd_dim_101,32,cashflow_Long Term Debt Issuance,balance_Accounts Payable,balance_Current Debt,695,svd_dim_50
0,-0.084914,-0.984096,0.268918,0.107418,-0.046604,-1.011947,-0.375769,0.000000e+00,2.815400e+09,0.017998,...,0.051443,-0.384085,-0.382449,0.050529,-0.734844,1.000000e+09,6.213000e+08,1.500000e+06,0.935146,-0.036939
1,0.011697,-0.408780,0.058946,-0.211146,-0.258639,-1.149698,0.105433,6.617404e+09,1.161558e+09,0.354260,...,-0.258659,0.259594,0.004463,-0.062224,-0.494972,0.000000e+00,2.166409e+09,5.948200e+07,0.983424,0.011989
2,-0.045409,0.379066,0.062867,-0.268551,-0.470384,-0.945055,1.394683,3.160000e+06,9.470280e+08,0.060547,...,-0.501833,-0.082250,-0.999183,0.172444,-0.779211,NaN,2.012461e+09,NaN,0.749328,-0.801159
3,0.057648,-0.725314,0.736935,-0.064104,-0.223336,-0.799952,0.159469,NaN,NaN,-0.076473,...,0.011499,-1.051099,-0.039436,-0.029329,-0.679992,NaN,NaN,NaN,0.727001,0.598416
4,0.012955,-0.828527,-0.378939,-0.294139,0.299100,-1.169436,-0.405682,1.254200e+09,6.876000e+08,0.077355,...,-0.182396,1.061694,1.390666,-0.036480,-0.481583,9.894000e+08,1.484000e+08,NaN,0.349011,-1.085175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,-0.099896,-0.163829,0.745106,-0.478851,-0.967331,-0.755338,0.922344,6.523050e+08,5.745470e+08,-0.000538,...,0.352982,-1.036170,2.143509,-0.118415,-0.211222,1.513807e+09,1.025200e+08,1.005000e+08,1.129584,-0.219964
843,0.017421,-0.883623,-0.034092,0.383691,0.312352,-0.093825,-0.196421,3.684000e+09,2.814000e+09,0.219506,...,-0.215960,-0.695535,0.292087,0.030085,-0.425754,4.026000e+09,NaN,4.838000e+09,0.546618,0.411675
844,0.026311,-0.129050,0.170490,-0.054193,-0.399259,-0.675335,0.057946,9.537000e+09,NaN,0.154962,...,0.047271,-0.103423,-0.755071,-0.001880,-0.450927,5.330000e+09,NaN,NaN,0.514164,-0.469090
845,-0.015391,-0.068218,0.163940,0.009989,-0.350510,-1.147584,0.601176,1.581000e+09,1.837400e+10,0.032089,...,0.135304,-0.156433,-0.379208,-0.044857,-0.130985,0.000000e+00,1.574000e+09,8.190000e+08,1.275327,0.384824


In [10]:
y

0      15.30
1      27.66
2      18.13
3      18.25
4      21.07
       ...  
842    28.98
843    18.73
844    21.70
845    30.41
846    19.73
Name: total_score, Length: 847, dtype: float64

In [11]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=100
)

In [ ]:
scorer = sklearn.metrics.get_scorer("neg_mean_squared_error")

# Initialize TPOT2 regressor with K-Fold cross-validation
est = tpot2.TPOTEstimatorSteadyState(
    n_jobs=6,
    cv=KFold(n_splits=5),  # 5-Fold cross-validation
    verbose=2,
    classification=False,
    scorers=[scorer],
    scorers_weights=[1],
    max_eval_time_seconds=60 * 10,
    max_time_seconds=60 * 90,
)

# Fit the model
est.fit(X_train, y_train)
print('Done fitting/training TPOT2 session.')


df_individuals = est.evaluated_individuals

# Convert the 'mean_squared_error' column to numeric, errors='coerce' will replace non-numeric with NaN
df_individuals['mean_squared_error'] = pd.to_numeric(df_individuals['mean_squared_error'], errors='coerce')

# Drop NaN values
filtered_df = df_individuals.dropna(subset=['mean_squared_error'])

# Sort the DataFrame by 'mean_squared_error' and get the top 10
top_10_mse = filtered_df.nlargest(10, 'mean_squared_error')

print(est.pareto_front)
print(top_10_mse)
print(est.get_params())



In [ ]:
top_10_mse

In [ ]:
top_30_mse = filtered_df.nlargest(30, 'mean_squared_error')


In [ ]:
# save the top 30 models to csv
top_30_mse.to_csv("../data/model_data/top_30_mse.csv")

In [ ]:
top_30_mse

In [12]:
import sys

sys.path.append("../")

In [13]:
from models.XGBoost.train_and_evaluate_model import train_and_evaluate_model
from models.XGBoost.tune_xgb_hyperparameters import tune_xgb_hyperparameters


In [ ]:
print(X_train.dtypes)


In [14]:
X_train = X_train.apply(pd.to_numeric, errors='ignore')


In [15]:
best_params = tune_xgb_hyperparameters(X_train, y_train, n_trials=30)


[I 2023-10-04 14:33:18,199] A new study created in memory with name: no-name-827e20e5-4c60-45b2-adb8-13acb2a5c227
[I 2023-10-04 14:34:42,408] Trial 0 finished with value: 5.141039159971211 and parameters: {'learning_rate': 0.012968149348245523, 'max_depth': 8, 'subsample': 0.6737816874766481, 'colsample_bytree': 0.7578570649509669, 'min_child_weight': 18}. Best is trial 0 with value: 5.141039159971211.
[I 2023-10-04 14:35:20,428] Trial 1 finished with value: 7.001739561173105 and parameters: {'learning_rate': 0.0010677992767126456, 'max_depth': 19, 'subsample': 0.149028769598958, 'colsample_bytree': 0.5119803140011495, 'min_child_weight': 9}. Best is trial 0 with value: 5.141039159971211.
[I 2023-10-04 14:37:12,585] Trial 2 finished with value: 5.544876382607322 and parameters: {'learning_rate': 0.044101081500749706, 'max_depth': 10, 'subsample': 0.5864591401556309, 'colsample_bytree': 0.26989308191738975, 'min_child_weight': 2}. Best is trial 0 with value: 5.141039159971211.
[I 2023-1

Best hyperparameters: {'learning_rate': 0.02998822459568964, 'max_depth': 5, 'subsample': 0.7095254982521659, 'colsample_bytree': 0.6113315193397809, 'min_child_weight': 13}
Best RMSE: 5.098858708449277


In [ ]:
# Best hyperparameters: {'learning_rate': 0.02998822459568964, 'max_depth': 5, 'subsample': 0.7095254982521659, 'colsample_bytree': 0.6113315193397809, 'min_child_weight': 13}
# Best RMSE: 5.098858708449277

In [16]:
import pickle

In [17]:
with open('../models/XGBoost/best_params_features_cleaned.pkl', 'wb') as f:
    pickle.dump(best_params, f)

In [ ]:
with open('../models/XGBoost/best_params_features_cleaned.pkl', 'rb') as f:
    best_params = pickle.load(f)

In [ ]:
best_params

In [18]:
trained_model, validation_rmse, feature_importances_df = train_and_evaluate_model(X_train, y_train, X_test, y_test, best_params)

Feature Importances:
               Feature  Importance
3                   69    0.012488
0      tfidf_svd_dim_4    0.010697
621                602    0.010175
637         svd_dim_29    0.008454
4                  287    0.007680
..                 ...         ...
453  tfidf_svd_dim_107    0.000000
449         svd_dim_27    0.000000
447   tfidf_svd_dim_54    0.000000
444                329    0.000000
797         svd_dim_50    0.000000

[798 rows x 2 columns]
RMSE: 4.877658008236885


In [ ]:
feature_importances_df

In [ ]:
filtered_feature_importances_df = feature_importances_df[feature_importances_df['Importance'] != 0]


In [ ]:
df

In [ ]:
filtered_feature_importances_df

In [ ]:
filtered_main_df = df.loc[:, filtered_feature_importances_df['Feature'].tolist() + ['total_score']]



In [ ]:
filtered_main_df

In [ ]:
filtered_main_df.to_csv("../data/ready_to_model/filtered_feature_importance_df.csv")

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import yfinance as yf
import pandas as pd


def fetch_data_for_row(named_tuple_row):
    ticker = "Unknown"  # Initialize with a default value
    year = "Unknown"    # Initialize with a default value
    
    try:
        ticker = named_tuple_row.ticker
        year = named_tuple_row.year

        yf_ticker = yf.Ticker(ticker)
        financials = yf_ticker.financials
        cashflow = yf_ticker.cashflow
        balance = yf_ticker.balance_sheet
        info = {k: v for k, v in yf_ticker.info.items() if isinstance(v, (int, float))}

        financials = financials.loc[:, pd.to_datetime(financials.columns).year == year].transpose()
        cashflow = cashflow.loc[:, pd.to_datetime(cashflow.columns).year == year].transpose()
        balance = balance.loc[:, pd.to_datetime(balance.columns).year == year].transpose()
        info_df = pd.DataFrame([info])

        financials.columns = 'financials_' + financials.columns.astype(str)
        cashflow.columns = 'cashflow_' + cashflow.columns.astype(str)
        balance.columns = 'balance_' + balance.columns.astype(str)
        info_df.columns = 'info_' + info_df.columns.astype(str)

        merged_data = pd.concat([financials, cashflow, balance, info_df], axis=1)
        merged_data['ticker'] = ticker
        merged_data['year'] = year

        return merged_data.reset_index(drop=True)
    except Exception as e:
        print(f"An error occurred in fetch_data_for_row for ticker: {ticker} and year: {year}. Error: {e}")
        return pd.DataFrame()



def fetch_and_merge_data(df):
    try:
        # Initialize an empty list to store fetched data
        fetched_data_list = []
        
        with ThreadPoolExecutor() as executor:
            fetched_data_list = list(executor.map(fetch_data_for_row, df.itertuples(index=False)))

        # Concatenate all the fetched data
        new_data = pd.concat([data.iloc[[0]] for data in fetched_data_list if not data.empty], ignore_index=True)

        # Debug: Print the shape and columns of new_data
        print(f"new_data shape: {new_data.shape}, columns: {new_data.columns}")

        # Merge new_data with df based on 'ticker' and 'year'
        final_df = pd.merge(df, new_data, on=['ticker', 'year'], how='left')

        return final_df
    except Exception as e:
        print(f"An error occurred in fetch_and_merge_data: {e}")
        return df  # Return the original DataFrame as a fallback




# Example usage
# df = pd.DataFrame({'ticker': ['AAPL', 'GOOGL'], 'year': [2020, 2021]})
# final_df = fetch_and_merge_data(df)


In [ ]:
df

In [ ]:
final_df = fetch_and_merge_data(df)

In [ ]:
df = final_df

In [ ]:
final_df.sample(10)

In [ ]:
# Calculate the percentage of missing values for each column
missing_percent = df_cleaned.isnull().mean() * 100

# Sort the columns by percentage of missing values in descending order
missing_percent_sorted = missing_percent.sort_values(ascending=False)

# Show the sorted series
print(missing_percent_sorted)


In [ ]:
nan_count = final_df.isna().sum()


In [ ]:
def remove_columns_with_nans(df, threshold=800):
    nan_count = final_df.isna().sum()
    columns_to_remove = nan_count[nan_count > threshold].index.tolist()
    df_cleaned = final_df.drop(columns=columns_to_remove)
    return df_cleaned

In [ ]:
df_cleaned = remove_columns_with_nans(df, threshold=300)


In [ ]:
df_cleaned



In [ ]:
df_cleaned.to_csv("../data/ready_to_model/df_cleaned_with_yfinance.csv")

In [ ]:
# Sorting columns by the number of NaN values (in descending order)
sorted_nan_count = nan_count.sort_values(ascending=False)

print(sorted_nan_count.value_counts())

In [ ]:
import pandas as pd

In [ ]:
df_topics = pd.read_csv("../data/ready_to_model/df_cleaned_with_yfinance.csv", index_col=0, parse_dates=True, sep=",")

In [ ]:
df_topics.head()